In [1]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np

In [2]:
root_path = Path('./data/')
results = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            print(os.path.join(subdir, file))
            _, query, params, seed = subdir.split('/')
            if query not in results:
                results[query] = {}

            if params not in results[query]:
                results[query][params] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results[query][params][seed] = res

data/badge/lr0.01_wd0.005/seed1/results.json
data/badge/lr0.01_wd0.005/seed2/results.json
data/badge/lr0.001_wd0.005/seed1/results.json
data/badge/lr0.001_wd0.005/seed3/results.json
data/badge/lr0.001_wd0.005/seed2/results.json
data/badge/lr0.001_wd0.05/seed1/results.json
data/badge/lr0.001_wd0.05/seed2/results.json
data/badge/lr0.001_wd0.0005/seed1/results.json
data/badge/lr0.001_wd0.0005/seed3/results.json
data/badge/lr0.001_wd0.0005/seed2/results.json
data/badge/lr0.01_wd0.0005/seed1/results.json
data/badge/lr0.01_wd0.0005/seed3/results.json
data/badge/lr0.01_wd0.0005/seed2/results.json
data/badge/lr0.01_wd0.05/seed1/results.json
data/badge/lr0.01_wd0.05/seed3/results.json
data/badge/lr0.01_wd0.05/seed2/results.json
data/badge/lr0.1_wd0.005/seed1/results.json
data/badge/lr0.1_wd0.005/seed3/results.json
data/badge/lr0.1_wd0.005/seed2/results.json
data/badge/lr0.1_wd0.0005/seed1/results.json
data/badge/lr0.1_wd0.0005/seed3/results.json
data/badge/lr0.1_wd0.0005/seed2/results.json
data

In [3]:
test_accuracys = {}

for query in results:
    test_accuracys[query] = {}
    for params in results[query]:
        test_accuracys[query][params] = []
        for seed in results[query][params]:
            # Find optimal run
            best_acc = -1
            for trial in results[query][params][seed]['ray-results']:
                if 'test_acc' in trial['res']:
                    if trial['res']['test_acc'] > best_acc:
                        best_acc = trial['res']['test_acc']
            test_accuracys[query][params].append(best_acc*100)

In [4]:
# Validate that each param group has 3 finished runs
for query in test_accuracys:
    for params in test_accuracys[query]:
        if len(test_accuracys[query][params]) != 3:
            print("Assertion: The run with query strategy "+str(query)+ " and parameters "+params+" has not had 3 finished runs yet!")

Assertion: The run with query strategy badge and parameters lr0.01_wd0.005 has not had 3 finished runs yet!
Assertion: The run with query strategy badge and parameters lr0.001_wd0.05 has not had 3 finished runs yet!


In [5]:
df = {}
for query in test_accuracys:
    df[query] = {}
    for params in test_accuracys[query]:
        df[query][params] = str(np.mean(test_accuracys[query][params]).round(4)) + ' +/- ' + str(np.std(test_accuracys[query][params]).round(4))
pd.DataFrame(df)

,badge,random
lr0.01_wd0.005,84.66 +/- 0.22,83.4367 +/- 0.0759
lr0.001_wd0.005,84.4 +/- 0.1393,83.2567 +/- 0.2207
lr0.001_wd0.05,84.285 +/- 0.105,83.0267 +/- 0.212
lr0.001_wd0.0005,84.4167 +/- 0.3389,82.9567 +/- 0.0736
lr0.01_wd0.0005,84.58 +/- 0.2844,83.01 +/- 0.1219
lr0.01_wd0.05,84.0433 +/- 0.5878,83.0767 +/- 0.213
lr0.1_wd0.005,84.51 +/- 0.3612,83.1867 +/- 0.147
lr0.1_wd0.0005,84.3333 +/- 0.319,83.3267 +/- 0.1841
lr0.1_wd0.05,NaN,83.07 +/- 0.3982
